In [1]:
import tensorflow as tf

C:\Users\hdc\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\hdc\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\hdc\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int1

In [2]:
sess = tf.InteractiveSession()

In [4]:
x = tf.placeholder("float", [None, 28*28])
y_ = tf.placeholder("float", [None, 10])

In [5]:
# 定义权重W和偏置bias
W = tf.Variable(tf.zeros([28*28, 10]))
b = tf.Variable(tf.zeros([10]))

In [6]:
sess.run(tf.global_variables_initializer())

In [8]:
y = tf.nn.softmax(tf.matmul(x, W)+b)

In [9]:
cross_entropy = -tf.reduce_sum(y_*tf.log(y))

In [10]:
# 这一行代码实际上是用来往计算图上添加一个新操作，其中包括计算梯度，计算每个参数的步长变化，并且计算出新的参数值
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

In [12]:
import input_data
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)
for i in range(1000):
    batch = mnist.train.next_batch(50)
    train_step.run(feed_dict={x: batch[0], y_: batch[1]})

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [13]:
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))

In [14]:
# tf.cast将bool数组投射到float数组，用于计算平均值
accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))

In [15]:
print(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9092


### 构建一个多层卷积网络

#### 权重初始化  
为了创建这个模型，我们需要创建大量的权重和偏置项。这个模型中的权重在初始化时应该加入少量的噪声来打破对称性以及避免0梯度。由于使用的ReLU激活函数$f(x)=\max(0,x)$,为了避免神经元节点输出恒为0的问题(dead neurons)。为了不在建立模型的时候反复做初始化操作，定义两个函数用于初始化。

In [16]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

#### 卷积和池化  
TensorFlow在卷积和池化上有很强的灵活性。在这个实例里，我们会一直使用vanilla版本，即卷积核使用1步长，0边距，保证输出和输入是同一个大小。池化使用$2 \times 2$大小的模板做max pooling。为了代码更灵活，我们把这部分抽象成一个函数。  
(1)same  
out_height=ceil(float(in_height))/float(strides[1])  
out_width=ceil(float(in_width))/float(strides[2])  
(2)valid  
out_height=ceil(float(in_height-filter_height+1))/float(strides[1])  
out_width=ceil(float(in_width-filter_width+1))/float(strides[2])  
————————————————  
版权声明：本文为CSDN博主「梦的漂流瓶」的原创文章，遵循 CC 4.0 BY-SA 版权协议，转载请附上原文出处链接及本声明。  
原文链接：https://blog.csdn.net/qq_32466233/article/details/81075288

In [23]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

#### 第一层卷积  
现在我们开始实现第一层卷积。它由一个卷积接一个max pooling完成。卷积在每个$5 \times 5$的patch中计算32个特征。卷积的权重张量形状是[5,5,1,32]#filter_height,filter_width,input_channels,output_channels。而对于每一个输出通道都有一个对应的偏置量。

In [18]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

为了用这一层，我们把$x$变成一个4d向量，其第2、第3维对应图片的宽、高，最后一维代表图片的颜色通道数。

In [19]:
x_image = tf.reshape(x, [-1, 28, 28, 1])

我们把x_image和权重向量进行卷积，加上偏置项，然后应用ReLU激活函数，最后进行max_pooling

In [24]:
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1)+b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

#### 第二层卷积  
为了构架一个更深的网络，我们会把几个类似的层堆叠起来。第二层中，每个$5 \times 5$的patch会得到64个特征。

In [26]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2)+b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

#### 密集连接层  
现在，图片尺寸减小到$7\times7$，我们加入一个有1024个神经元的全连接层，用于处理整个图片。我们把池化层输出的张量reshape成一维向量，乘上权重矩阵，加上偏置，然后对其使用ReLU.

In [28]:
W_fc1 = weight_variable([7*7*64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1)+b_fc1)

#### Dropout  
为了减少过拟合，我们在输出层之前加入dropout, 我们用一个placeholder来代表一个神经元的输出在dropout中保持不变的概率。这样我们可以在训练过程中启用dropout，在测试过程中关闭dropout。TensorFlow的tf.nn.dropout操作除了可以屏蔽神经元的输出外，还会自动处理神经元输出值的scale。所以用dropout的时候可以不用考虑scale。

In [29]:
keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


#### 输出层  
最后，我们添加一个softmax层

In [30]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

#### 训练和评估模型  
用更加复杂的Adam优化器做梯度最速下降，在feed_dict中加入额外的keep_prob来控制dropout比例，然后每100次迭代输出一次日志.

In [ ]:
cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))
sess.run(tf.global_variables_initializer())

for i in range(20000):
    batch = mnist.train.next_batch(50)
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
        print("step: %d, train_accuracy: %.6f"%(i, train_accuracy))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

print("test accuracy: %.6f"%accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

step: 0, train_accuracy: 0.120000
step: 100, train_accuracy: 0.800000
step: 200, train_accuracy: 0.880000
step: 300, train_accuracy: 0.900000
step: 400, train_accuracy: 0.940000
step: 500, train_accuracy: 0.860000
step: 600, train_accuracy: 0.920000
step: 700, train_accuracy: 0.960000
step: 800, train_accuracy: 0.820000
step: 900, train_accuracy: 0.880000
step: 1000, train_accuracy: 0.920000
step: 1100, train_accuracy: 0.940000
step: 1200, train_accuracy: 1.000000
step: 1300, train_accuracy: 0.940000
step: 1400, train_accuracy: 0.920000
step: 1500, train_accuracy: 0.940000
step: 1600, train_accuracy: 0.920000
step: 1700, train_accuracy: 0.940000
step: 1800, train_accuracy: 0.980000
step: 1900, train_accuracy: 0.940000
step: 2000, train_accuracy: 0.960000
step: 2100, train_accuracy: 1.000000
step: 2200, train_accuracy: 0.980000
step: 2300, train_accuracy: 1.000000
step: 2400, train_accuracy: 0.980000
step: 2500, train_accuracy: 0.980000
step: 2600, train_accuracy: 0.960000
step: 2700, t